<a href="https://colab.research.google.com/github/Madushani-Weerasekara/D11_Bidirectional-LSTM_fake-news-classifier/blob/main/D11_Bidirectional_LSTM_fake_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake News Classifier using Bidirectional LSTM RNN

In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('/content/fake_or_real_news.csv', engine='python', error_bad_lines=False)

<ipython-input-7-af2d7ed5b0b4>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/fake_or_real_news.csv', engine='python', error_bad_lines=False)
Skipping line 1701: unexpected end of data


In [8]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [10]:
df.shape

(1699, 4)

In [11]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [13]:
# Get Independent features
x = df.drop('label', axis=1)

In [14]:
# Dependent feature
y = df['label']

In [15]:
# Check wheather dataset is balance or not
y.value_counts()

FAKE    872
REAL    827
Name: label, dtype: int64

In [16]:
import tensorflow as tf

In [17]:
tf.__version__

'2.15.0'

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional

In [22]:
#Vocabulary size
voc_size = 5000

In [23]:
messages = x.copy()

In [25]:
messages.head()

,Unnamed: 0,title,text
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...


In [26]:
messages.shape

(1699, 3)